In [1]:
import math
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import jax
import jax.numpy as jnp
import jax.scipy.stats as stats
from jax import jacfwd, jacrev
from jax import vmap, grad, jit, random
from jax.tree_util import tree_map, tree_flatten, tree_unflatten, tree_leaves

from flax import nnx
import optax
import pcax

from tqdm.notebook import tqdm
from tqdm.notebook import trange

In [2]:
import os
os.chdir('../methods')
import lsci, supr, conf, uqno
os.chdir('../energy')

In [3]:
def split_data(data, lag, horizon):
    horizon = horizon-1
    y_t = data[(lag + horizon):][:,None]
    x_t = np.stack([data[(lag-i-1):(-(i+1+horizon))] for i in range(lag)], axis = 1)
    return x_t.copy(), y_t.copy()

In [6]:
# https://www.eia.gov/electricity/gridmonitor/dashboard/electric_overview/US48/US48
import pandas as pd
energy = pd.read_excel('../data/tx_energy.xlsx')
# energy = np.array(energy)

In [12]:
energy_curves =  energy.groupby('Local date')['Demand'].apply(list).reset_index()

In [21]:
energy_curves2 = np.array(energy_curves)

In [111]:
dts = energy_curves2[:,0]
fns = energy_curves2[:,1]

In [126]:
# transform
fns = energy_curves2[:,1]
fns = [np.array(f) for f in fns]
fns = [jax.image.resize(f, 24, 'bicubic') for f in fns]
fns = np.array(fns)

# backfill missing
missing = jnp.sum(jnp.isnan(fns), axis = 1) > 0
for i in range(fns.shape[0]):
    if missing[i]:
        fns[i] = fns[i-1]

# rescale
fns = (fns - jnp.mean(fns, axis = 0)[None,:]) / jnp.std(fns, axis = 0)[None,:]
fns = jnp.array(fns)[:,None,None,:]

In [127]:
fns.shape

(3522, 1, 1, 24)